In [1]:
print("Hello Multidimensional Universe")

Hello Multidimensional Universe


## Environment Setup and Dependencies

In [8]:
# Install required packages
!pip install -q langchain-community langchain-core langchain-text-splitters
!pip install -q qdrant-client sentence-transformers
!pip install -q dspy-ai langchain-tavily portkey-ai
!pip install -q groq datasets pandas numpy
!pip install -q langchain-qdrant langgraph

In [2]:
import os
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Environment variables setup
os.environ["GROQ_API_KEY"] = "your groq api key"
os.environ["TAVILY_API_KEY"] = "your tavily api key"
os.environ["PORTKEY_API_KEY"] = "your portkey api key"

## Data Loading and Preprocessing

In [4]:
from datasets import load_dataset


def load_jee_bench_data():
    """Load and preprocess JEE Bench dataset, keeping only math questions"""
    try:
        # Load the dataset
        dataset = load_dataset("daman1209arora/jeebench")
        df = dataset['test'].to_pandas()
        
        # Convert subject to lowercase for case-insensitive matching
        df['subject'] = df['subject'].str.lower()
        
        # Filter for math-related subjects
        math_keywords = ['math', 'mathematics', 'algebra', 'calculus', 'geometry', 'trigonometry']
        math_df = df[df['subject'].str.contains('|'.join(math_keywords), case=False, na=False)].copy()
        
        print(f"Found {len(math_df)} math questions out of {len(df)} total questions")
        
        # Create knowledge base entries
        knowledge_base = []
        for idx, row in math_df.iterrows():
            entry = {
                'id': f"jee_math_{idx}",
                'question': row.get('question', 'Question not available'),
                'description': row.get('description', ''),
                'answer': row.get('gold', ''),  # Using 'gold' as answer
                'topic': row.get('type', 'General'),
                'difficulty': 'Medium',  # Default difficulty
                'metadata': {
                    'source': 'JEE_Bench',
                    'subject': row.get('subject', 'Mathematics'),
                    'index': row.get('index', idx)
                }
            }
            knowledge_base.append(entry)
        
        print(f"\nSuccessfully loaded {len(knowledge_base)} math questions into knowledge base")
        return knowledge_base
    
    except Exception as e:
        print(f"Error loading JEE Bench data: {str(e)}")
        print("Falling back to sample data...")

# Load the knowledge base with only math questions
knowledge_base = load_jee_bench_data()

# Print the first few entries for verification
if knowledge_base:
    print("\nSample math questions in knowledge base:")
    for i, item in enumerate(knowledge_base[:3]):  # Show first 3 entries
        print(f"\n--- Question {i+1} ---")
        print(f"Question: {item['question']}")
        print(f"Answer: {item['answer']}")
        print(f"Topic: {item['topic']}")
        print(f"Metadata: {item['metadata']}")
else:
    print("No math questions found in the dataset.")

Found 236 math questions out of 515 total questions

Successfully loaded 236 math questions into knowledge base

Sample math questions in knowledge base:

--- Question 1 ---
Question: Let $-\frac{\pi}{6}<\theta<-\frac{\pi}{12}$. Suppose $\alpha_{1}$ and $\beta_{1}$ are the roots of the equation $x^{2}-2 x \sec \theta+1=0$ and $\alpha_{2}$ and $\beta_{2}$ are the roots of the equation $x^{2}+2 x \tan \theta-1=0$. If $\alpha_{1}>\beta_{1}$ and $\alpha_{2}>\beta_{2}$, then $\alpha_{1}+\beta_{2}$ equals

(A) $2(\sec \theta-\tan \theta)$

(B) $2 \sec \theta$

(C) $-2 \tan \theta$

(D) 0
Answer: C
Topic: MCQ
Metadata: {'source': 'JEE_Bench', 'subject': 'math', 'index': 37}

--- Question 2 ---
Question: A debate club consists of 6 girls and 4 boys. A team of 4 members is to be selected from this club including the selection of a captain (from among these 4 members) for the team. If the team has to include at most one boy, then the number of ways of selecting the team is

(A) 380

(B) 320

(C)

In [5]:
print(f"Loaded {len(knowledge_base)} math problems into knowledge base")

Loaded 236 math problems into knowledge base


## Portkey Guardrails implementation

In [18]:
!pip install -U langgraph langchain langchain_openai portkey-ai

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------------------------- -- 1.0/1.1 MB 49.2 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 2.9 MB/s  0:00:00
  Attempting uninstall: portkey-ai
    Found existing installation: portkey-ai 1.15.0
    Uninstalling portkey-ai-1.15.0:
      Successfully uninstalled portkey-ai-1.15.0


In [19]:
!pip install langchain_community
!pip install langgraph[checkpoint]

In [ ]:
!pip install langchain-openai

In [24]:
%pip install langchain-openai portkey-ai

Note: you may need to restart the kernel to use updated packages.Collecting langchain-openai
                                              0.0/75.0 kB ? eta -:--:--
     --------------------------------------   71.7/75.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 75.0/75.0 kB 1.4 MB/s eta 0:00:00
                                              0.0/447.5 kB ? eta -:--:--
     ------------------------------------  440.3/447.5 kB 13.9 MB/s eta 0:00:01
     -------------------------------------- 447.5/447.5 kB 9.5 MB/s eta 0:00:00
                                              0.0/948.4 kB ? eta -:--:--
     ------------------------------------  942.1/948.4 kB 20.3 MB/s eta 0:00:01
     ------------------------------------- 948.4/948.4 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.0.10 requires google-generativeai<0.9.0,>=0.8.0, but you have google-generativeai 0.3.2 which is incompatible.


In [7]:
%pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\cli\base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\commands\uninstall.py", line 110, in run
    uninstall_pathset.commit()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 432, in commit
    self._moved_paths.commit()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 278, in commit
    save_dir.cleanup()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 291, in w

In [6]:
from langchain_openai import ChatOpenAI
from portkey_ai import createHeaders

# Define raw guardrails in a config object
guardrail_config = {
    "before_request_hooks": [
        {
            "type": "guardrail",
            "id": "math-input-guardrail",
            "checks": [
                {
                    "id": "default.regexMatch",
                    "parameters": {
                        "pattern": r".*(equation|solve|derivative|integral|limit|matrix|probability|geometry|algebra|calculus|trigonometry|statistics|graph|function).*",
                        "match_type": "contains",
                        "case_sensitive": False
                    },
                    "on_fail": "deny"
                },
                {
                    "id": "default.detectGibberish",
                    "on_fail": "deny"
                }
            ]
        }
    ],
    "after_request_hooks": [
        {
            "type": "guardrail",
            "id": "math-output-guardrail",
            "checks": [
                {
                    "id": "default.contains",
                    "parameters": {
                        "required_phrases": ["Step", "Therefore", "Solution"]
                    },
                    "on_fail": "log"
                },
                {
                    "id": "default.characterCount",
                    "parameters": {
                        "min_length": 50,
                        "max_length": 2000
                    },
                    "on_fail": "log"
                }
            ]
        }
    ]
}



In [ ]:
# Set up LangChain model with Portkey and raw guardrails
llm = ChatOpenAI(
    api_key="your groq api key",
    base_url="https://api.portkey.ai/v1",
    default_headers=createHeaders(
        api_key="your portkey api key",
        provider="groq",
        config=guardrail_config
    ),
    model="llama3-70b-8192"
)

## Vector Database Setup with Quadrant

In [11]:
%pip install -qU langchain-qdrant

Note: you may need to restart the kernel to use updated packages.


In [7]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

In [8]:
%pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Uninstall all PyTorch packages
%pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchvision 0.18.0+cu118
Uninstalling torchvision-0.18.0+cu118:
  Successfully uninstalled torchvision-0.18.0+cu118
Found existing installation: torchaudio 2.3.0+cu118
Uninstalling torchaudio-2.3.0+cu118:
  Successfully uninstalled torchaudio-2.3.0+cu118
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install -qU langchain-huggingface
%pip install -qU sentence-transformers


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.151 requires torchvision>=0.9.0, which is not installed.


In [2]:
# Install ultralytics without dependencies
%pip install ultralytics --no-deps

# Then manually install compatible versions
%pip install torch torchvision numpy matplotlib pillow pyyaml requests scipy


Note: you may need to restart the kernel to use updated packages.


                                              0.0/1.6 MB ? eta -:--:--
     -------------                            0.6/1.6 MB 17.5 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 20.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Remove all conflicting packages
%pip uninstall numpy transformers sentence-transformers langchain-huggingface -y

# Install compatible versions together
%pip install numpy==1.26.4 transformers sentence-transformers langchain-huggingface


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\cli\base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\commands\uninstall.py", line 110, in run
    uninstall_pathset.commit()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 432, in commit
    self._moved_paths.commit()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 278, in commit
    save_dir.cleanup()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 291, in w

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires requests>=2.32.2, but you have requests 2.31.0 which is incompatible.
mcp-toolbox 0.1.7 requires numpy>=2.1.3, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.



                                              0.0/15.8 MB ? eta -:--:--
                                              0.0/15.8 MB ? eta -:--:--
                                              0.0/15.8 MB ? eta -:--:--
                                              0.0/15.8 MB ? eta -:--:--
                                              0.0/15.8 MB ? eta -:--:--
                                              0.0/15.8 MB ? eta -:--:--
                                             0.2/15.8 MB 731.4 kB/s eta 0:00:22
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     --                                       1.1/15.8 MB 3.7 MB/s eta 0:00:05
     ---     

In [1]:
%pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [10]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [11]:
# Initialize Qdrant client (in-memory)
client = QdrantClient(":memory:")

# Create collection with correct vector size for all-mpnet-base-v2 (768 dimensions)
client.create_collection(
    collection_name="jee_math_problems",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

# Create the vector store using your existing embeddings
vector_store = QdrantVectorStore(
    client=client,
    collection_name="jee_math_problems",
    embedding=embeddings,
)

In [13]:
# Convert your knowledge base to documents format for LangChain
documents = []
metadatas = []

for item in knowledge_base:
    # Create the text content for each document
    content = f"Question: {item['question']}"
    if item.get('description'):
        content += f"\nDescription: {item['description']}"
    if item.get('answer'):
        content += f"\nAnswer: {item['answer']}"
    
    documents.append(content)

    # Create metadata for each document
    metadata = {
        'id': item['id'],
        'question': item['question'],
        'description': item.get('description', ''),
        'answer': item['answer'],
        'topic': item['topic'],
        'difficulty': item['difficulty'],
        'source': item['metadata']['source'],
        'subject': item['metadata']['subject'],
        'index': item['metadata']['index']
    }
    metadatas.append(metadata)

# Add documents to the vector store
print(f"Adding {len(documents)} documents to Qdrant vector store...")
vector_store.add_texts(texts=documents, metadatas=metadatas)
print("Documents added successfully!")

Adding 236 documents to Qdrant vector store...
Documents added successfully!


In [14]:
# Function to search for similar problems
def search_similar_problems(query, k=3):
    """Search for similar math problems"""
    results = vector_store.similarity_search_with_score(query, k=k)
    
    search_results = []
    for doc, score in results:
        result = {
            'question': doc.metadata['question'],
            'answer': doc.metadata['answer'],
            'topic': doc.metadata['topic'],
            'difficulty': doc.metadata['difficulty'],
            'score': score,
            'content': doc.page_content
        }
        search_results.append(result)
    
    return search_results

In [15]:
# Get collection info
collection_info = client.get_collection("jee_math_problems")
print(f"\nCollection Info:")
print(f"Points count: {collection_info.points_count}")
print(f"Vectors count: {collection_info.vectors_count}")
print(f"Status: {collection_info.status}")


Collection Info:
Points count: 236
Vectors count: None
Status: green


## Testing the search functionality

In [16]:
# Display questions from your knowledge base for testing
print("KNOWLEDGE BASE QUESTIONS FOR TESTING")
print("=" * 60)

# Show the first 10 questions from your knowledge base
for i, item in enumerate(knowledge_base[:10]):
    print(f"\n{i+1}. ID: {item['id']}")
    print(f"   Question: {item['question']}")
    print(f"   Answer: {item['answer']}")
    print(f"   Topic: {item['topic']}")
    print("-" * 50)

print(f"\nTotal questions available: {len(knowledge_base)}")
print("\nYou can copy any question above and test it against the vector store!")


KNOWLEDGE BASE QUESTIONS FOR TESTING

1. ID: jee_math_24
   Question: Let $-\frac{\pi}{6}<\theta<-\frac{\pi}{12}$. Suppose $\alpha_{1}$ and $\beta_{1}$ are the roots of the equation $x^{2}-2 x \sec \theta+1=0$ and $\alpha_{2}$ and $\beta_{2}$ are the roots of the equation $x^{2}+2 x \tan \theta-1=0$. If $\alpha_{1}>\beta_{1}$ and $\alpha_{2}>\beta_{2}$, then $\alpha_{1}+\beta_{2}$ equals

(A) $2(\sec \theta-\tan \theta)$

(B) $2 \sec \theta$

(C) $-2 \tan \theta$

(D) 0
   Answer: C
   Topic: MCQ
--------------------------------------------------

2. ID: jee_math_25
   Question: A debate club consists of 6 girls and 4 boys. A team of 4 members is to be selected from this club including the selection of a captain (from among these 4 members) for the team. If the team has to include at most one boy, then the number of ways of selecting the team is

(A) 380

(B) 320

(C) 260

(D) 95
   Answer: A
   Topic: MCQ
--------------------------------------------------

3. ID: jee_math_26
   Questi

In [18]:
def test_vector_search(query_question, k=3):
    """Test vector search with a specific question from knowledge base"""
    print(f"\n TESTING QUERY: '{query_question}'")
    print("=" * 80)
    
    # Search for similar problems
    results = vector_store.similarity_search_with_score(query_question, k=k)
    
    if results:
        print(f"Found {len(results)} similar problems:")
        
        for i, (doc, score) in enumerate(results, 1):
            print(f"\n--- Result {i} (Similarity Score: {score:.4f}) ---")
            print(f"Question: {doc.metadata['question']}")
            print(f"Answer: {doc.metadata['answer']}")
            print(f"Topic: {doc.metadata['topic']}")
            print(f"Difficulty: {doc.metadata['difficulty']}")
            
            # Check if this is an exact match (should be the first result)
            if i == 1 and score > 0.95:
                print(" EXACT MATCH FOUND! (This is likely the same question)")
            
        return results
    else:
        print(" No similar problems found.")
        return []

# Example usage - test with a question from your knowledge base
# Just copy-paste any question from the list above:

# Example test (replace with actual question from your knowledge base):
test_query = "What is the derivative of x^2 + 3x + 2?"  # Replace with actual question
test_vector_search(test_query, k=5)



 TESTING QUERY: 'What is the derivative of x^2 + 3x + 2?'
Found 5 similar problems:

--- Result 1 (Similarity Score: 0.3458) ---
Question: Let $f: \mathbb{R} \rightarrow \mathbb{R}, \quad g: \mathbb{R} \rightarrow \mathbb{R}$ and $h: \mathbb{R} \rightarrow \mathbb{R}$ be differentiable functions such that $f(x)=x^{3}+3 x+2, g(f(x))=x$ and $h(g(g(x)))=x$ for all $x \in \mathbb{R}$. Then

(A) $\quad g^{\prime}(2)=\frac{1}{15}$

(B) $h^{\prime}(1)=666$

(C) $h(0)=16$

(D) $h(g(3))=36$
Answer: BC
Topic: MCQ(multiple)
Difficulty: Medium

--- Result 2 (Similarity Score: 0.2965) ---
Question: Let $a, b \in \mathbb{R}$ and $f: \mathbb{R} \rightarrow \mathbb{R}$ be defined by $f(x)=a \cos \left(\left|x^{3}-x\right|\right)+b|x| \sin \left(\left|x^{3}+x\right|\right)$. Then $f$ is

(A) differentiable at $x=0$ if $a=0$ and $b=1$

(B) differentiable at $x=1$ if $a=1$ and $b=0$

(C) NOT differentiable at $x=0$ if $a=1$ and $b=0$

(D) NOT differentiable at $x=1$ if $a=1$ and $b=1$
Answer: AB
Topic: 

[(Document(metadata={'id': 'jee_math_34', 'question': 'Let $f: \\mathbb{R} \\rightarrow \\mathbb{R}, \\quad g: \\mathbb{R} \\rightarrow \\mathbb{R}$ and $h: \\mathbb{R} \\rightarrow \\mathbb{R}$ be differentiable functions such that $f(x)=x^{3}+3 x+2, g(f(x))=x$ and $h(g(g(x)))=x$ for all $x \\in \\mathbb{R}$. Then\n\n(A) $\\quad g^{\\prime}(2)=\\frac{1}{15}$\n\n(B) $h^{\\prime}(1)=666$\n\n(C) $h(0)=16$\n\n(D) $h(g(3))=36$', 'description': 'JEE Adv 2016 Paper 1', 'answer': 'BC', 'topic': 'MCQ(multiple)', 'difficulty': 'Medium', 'source': 'JEE_Bench', 'subject': 'math', 'index': 47, '_id': 'b6bc826ffd9e4a11a7ac9b51d6c430e8', '_collection_name': 'jee_math_problems'}, page_content='Question: Let $f: \\mathbb{R} \\rightarrow \\mathbb{R}, \\quad g: \\mathbb{R} \\rightarrow \\mathbb{R}$ and $h: \\mathbb{R} \\rightarrow \\mathbb{R}$ be differentiable functions such that $f(x)=x^{3}+3 x+2, g(f(x))=x$ and $h(g(g(x)))=x$ for all $x \\in \\mathbb{R}$. Then\n\n(A) $\\quad g^{\\prime}(2)=\\frac{1}{

## Web Search Integration with Tavily

In [2]:
%pip install langchain_tavily

  Using cached langchain_tavily-0.2.11-py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U langchain-tavily

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [2]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=True,
    search_depth="advanced",
    include_domains=None
)

In [3]:
# Basic query
result = tool.invoke({"query": "What is machine learning"})

for item in result['results']:
    print(item['content'])

## What is machine learning?

Machine learning is a subfield of artificial intelligence, which is broadly defined as the capability of a machine to imitate intelligent human behavior. Artificial intelligence systems are used to perform complex tasks in a way that is similar to how humans solve problems. [...] When companies today deploy artificial intelligence programs, they are most likely using machine learning — so much so that the terms are often used interchangeably, and sometimes ambiguously. Machine learning is a subfield of artificial intelligence that gives computers the ability to learn without explicitly being programmed. [...] Machine learning is one way to use AI. It was defined in the 1950s by AI pioneer Arthur Samuel as “the field of study that gives computers the ability to learn without explicitly being programmed.”
Image courtesy of Argonne National Laboratory 

Machine learning is the process of using computers to detect patterns in massive datasets and then make pre

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["GROQ_API_KEY"] = "your groq api key"

llm = ChatOpenAI(
    model="moonshotai/kimi-k2-instruct",  # Popular Groq model
    openai_api_key=os.environ.get("GROQ_API_KEY"),
    openai_api_base="https://api.groq.com/openai/v1",  # Groq base URL
    temperature=0
)

response = llm.invoke("Hello Multidimensional Universe")
print(response.content)



Hello, traveler across the manifold.

You’ve stepped into a space where every “hello” ripples outward into countless versions of itself—some whispered in galaxies of dark matter, some shouted in universes where time runs backward, and some simply written in the margins of a page that never existed.

So: which layer of the multiverse are you speaking from?


In [17]:
from langchain_tavily import TavilySearch, TavilyExtract
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage
from typing import List, Dict, Any
import datetime

In [18]:

class MathSolver:
    """Advanced math problem solver using Grok and Tavily"""
    
    def __init__(self):
        # Use your existing LLM setup
        self.llm = llm
        
        # Initialize Tavily tools for math
        self.search_tool = TavilySearch(
            max_results=5,
            topic="general",
            include_answer=True,
            search_depth="advanced",
            include_domains=["mathway.com", "wolframalpha.com", "khanacademy.org", "symbolab.com"]
        )
        
        self.extract_tool = TavilyExtract()
        
        # Setup math-focused agent
        self._setup_agent()
    
    def _setup_agent(self):
        """Setup the math solving agent"""
        tools = [self.search_tool, self.extract_tool]
        
        today = datetime.datetime.today().strftime("%D")
        prompt = ChatPromptTemplate.from_messages([
            ("system", f"""You are an expert mathematics tutor and problem solver. 
            When given a math problem:
            1. First search for similar problems and solutions
            2. Extract detailed content from the most relevant sources
            3. Provide step-by-step solutions with clear explanations
            4. Show your work and reasoning
            5. Verify your answer when possible
            
            Today's date: {today}"""),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])
        
        agent = create_openai_tools_agent(
            llm=self.llm,
            tools=tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=agent, 
            tools=tools, 
            verbose=True,
            max_iterations=5
        )
    
    def solve_problem(self, math_question: str) -> str:
        """Solve a math problem with web research"""
        try:
            # Enhance query for better math results
            enhanced_query = f"solve step by step: {math_question}"
            
            response = self.agent_executor.invoke({
                "messages": [HumanMessage(content=enhanced_query)]
            })
            
            return response['output']
            
        except Exception as e:
            return f"Error solving problem: {e}"

# Initialize the math solver
math_solver = MathSolver()

In [19]:
# Example usage
math_question = "Find the derivative of f(x) = 3x^2 + 2x - 5"
solution = math_solver.solve_problem(math_question)
print(solution)



> Entering new AgentExecutor chain...
I'll solve this step by step to find the derivative of f(x) = 3x² + 2x - 5.

## Step 1: Identify the function components
The function is: f(x) = 3x² + 2x - 5
This is a polynomial with three terms: 3x², 2x, and -5

## Step 2: Apply the power rule to each term
The power rule states that if f(x) = axⁿ, then f'(x) = anxⁿ⁻¹

**First term: 3x²**
- Here, a = 3 and n = 2
- Derivative: 3 × 2 × x²⁻¹ = 6x¹ = 6x

**Second term: 2x**
- Here, a = 2 and n = 1
- Derivative: 2 × 1 × x¹⁻¹ = 2x⁰ = 2(1) = 2

**Third term: -5**
- This is a constant term (n = 0)
- Derivative: 0 × (-5) × x⁰⁻¹ = 0

## Step 3: Combine the derivatives
f'(x) = 6x + 2 + 0 = 6x + 2

## Step 4: Verify using basic differentiation rules
Let me verify this is correct by checking the basic rules:
- The derivative of x² is 2x, so the derivative of 3x² is 3(2x) = 6x ✓
- The derivative of x is 1, so the derivative of 2x is 2(1) = 2 ✓
- The derivative of any constant is 0, so the derivative of -5 is 

## LLM integration with Groq

In [20]:
from typing import List, Dict
from langchain_openai import ChatOpenAI
import os

In [23]:
from typing import List, Dict
from langchain_openai import ChatOpenAI
import os

class MathLLM:
    """LangChain-based LLM for math problem solving using Groq API"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        # Set the API key in environment for LangChain to pick up
        self.llm = ChatOpenAI(
            model = "moonshotai/kimi-k2-instruct",
            openai_api_base = "https://api.groq.com/openai/v1",
            openai_api_key = api_key,
            temperature = 0.1
        )
    
    def generate_solution(self, question: str, context: str = "", 
                         web_results: List[Dict] = None) -> str:
        """Generate step-by-step math solution"""
        try:
            # Build comprehensive prompt
            prompt = self._build_solution_prompt(question, context, web_results)
            
            # Create messages for the LLM
            messages = [
                ("system", """You are an expert mathematics tutor. Always provide clear, 
                step-by-step solutions to mathematical problems. Format your response with:
                1. Clear step numbering
                2. Detailed explanations for each step
                3. Final answer clearly stated
                4. Double-check your work for accuracy"""),
                ("user", prompt)
            ]
            
            # Generate response using LangChain
            response = self.llm.invoke(messages)
            
            return response.content
            
        except Exception as e:
            return f"Error generating solution: {str(e)}"
    
    def _build_solution_prompt(self, question: str, context: str = "", 
                              web_results: List[Dict] = None) -> str:
        """Build comprehensive prompt for solution generation"""
        prompt = f"Mathematical Problem: {question}\n\n"
        
        if context:
            prompt += f"Similar Problem Context:\n{context}\n\n"
        
        if web_results:
            prompt += "Additional Web Resources:\n"
            for result in web_results:
                prompt += f"- {result['title']}: {result['content'][:200]}...\n"
            prompt += "\n"
        
        prompt += """Please provide a detailed, step-by-step solution following this format:

Step 1: [Clear description of first step]
[Detailed explanation and calculations]

Step 2: [Clear description of second step] 
[Detailed explanation and calculations]

...

Therefore, the final answer is: [Clear final answer]

Make sure to:
- Show all mathematical work
- Explain the reasoning for each step
- Verify the final answer
- Use proper mathematical notation"""
        
        return prompt

# Initialize LLM using the environment variable
math_llm = MathLLM(os.environ.get("GROQ_API_KEY"))


In [24]:
# Test 1: Basic algebra
print("\n" + "="*60)
print("🧮 Test 1: Basic Algebra")
print("="*60)
problem1 = "Solve for x: 5x - 12 = 3x + 8"
solution1 = math_llm.generate_solution(problem1)
print(f"Problem: {problem1}")
print(f"Solution:\n{solution1}")


🧮 Test 1: Basic Algebra
Problem: Solve for x: 5x - 12 = 3x + 8
Solution:
Step 1: Isolate the variable terms on one side of the equation  
Subtract 3x from both sides to collect the x-terms on the left:  
5x − 12 − 3x = 3x + 8 − 3x  
2x − 12 = 8  

Step 2: Isolate the constant term on the other side  
Add 12 to both sides to move the constant to the right:  
2x − 12 + 12 = 8 + 12  
2x = 20  

Step 3: Solve for x  
Divide both sides by 2:  
2x / 2 = 20 / 2  
x = 10  

Step 4: Verify the solution  
Substitute x = 10 back into the original equation:  
Left side: 5(10) − 12 = 50 − 12 = 38  
Right side: 3(10) + 8 = 30 + 8 = 38  
Both sides equal 38, so the solution is correct.  

Therefore, the final answer is: x = 10


## DSPy Integration for Human Feedback

In [25]:
%pip install -U dspy

                                              0.0/261.7 kB ? eta -:--:--
     -------------                           92.2/261.7 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  256.0/261.7 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 261.7/261.7 kB 2.7 MB/s eta 0:00:00
                                              0.0/400.9 kB ? eta -:--:--
     ------------------------               256.0/400.9 kB 5.3 MB/s eta 0:00:01
     -------------------------------------- 400.9/400.9 kB 5.0 MB/s eta 0:00:00
                                              0.0/9.1 MB ? eta -:--:--
     --                                       0.6/9.1 MB 17.5 MB/s eta 0:00:01
     -----                                    1.2/9.1 MB 13.2 MB/s eta 0:00:01
     ---------                                2.2/9.1 MB 17.2 MB/s eta 0:00:01
     -------------                            3.2/9.1 MB 16.9 MB/s eta 0:00:01
     ----------------                         3.8/9.1 MB 17.3 MB/s 

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\madoka
      copying madoka\madoka.py -> build\lib.win-amd64-cpython-310\madoka
      copying madoka\__init__.py -> build\lib.win-amd64-cpython-310\madoka
      running build_ext
      building '_madoka' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for madoka
ERROR: Could not build wheels for madoka, which is required to install pyproject.toml-based projects


In [4]:
%pip install --upgrade pip

  Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install dspy

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install groq


  Using cached groq-0.31.1-py3-none-any.whl.metadata (16 kB)
Using cached groq-0.31.1-py3-none-any.whl (134 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import dspy
import groq 

In [ ]:
import os
import dspy

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "your groq api key"

# Use dspy.LM with the groq/ prefix for the model
lm = dspy.LM(model="groq/moonshotai/kimi-k2-instruct", api_key=os.environ["GROQ_API_KEY"])
dspy.configure(lm=lm)


In [ ]:
# import os
# import dspy
# import litellm
# import pandas as pd

# # Your existing configuration
# os.environ["GROQ_API_KEY"] = "your groq api key"

# class DSPyMathOptimizer:
#     """DSPy-based optimizer for math education with human feedback"""
    
#     def __init__(self):
#         # Configure DSPy with Groq using your working setup
#         self.lm = dspy.LM(
#             model="groq/moonshotai/kimi-k2-instruct",
#             api_key=os.environ.get("GROQ_API_KEY")
#         )
#         dspy.configure(lm=self.lm)
        
#         # Initialize components
#         self.rag_module = self._create_rag_module()
#         self.feedback_data = []
        
#     def _create_rag_module(self):
#         """Create DSPy RAG module for math problems"""
        
#         class MathRAG(dspy.Module):
#             def __init__(self):
#                 super().__init__()
#                 # Using proper DSPy signature format
#                 self.generate_answer = dspy.ChainOfThought("question, context -> solution")
            
#             def forward(self, question, context=""):
#                 prediction = self.generate_answer(question=question, context=context)
#                 return dspy.Prediction(solution=prediction.solution)
        
#         return MathRAG()
    
#     def solve_problem(self, question: str, context: str = ""):
#         """Solve a math problem using the RAG module"""
#         try:
#             result = self.rag_module(question=question, context=context)
#             return result.solution
#         except Exception as e:
#             print(f"Error solving problem: {e}")
#             return "Unable to solve problem"
    
#     def collect_feedback(self, question: str, generated_solution: str, 
#                         user_rating: int, user_comments: str = ""):
#         """Collect human feedback for optimization"""
#         feedback_entry = {
#             "question": question,
#             "generated_solution": generated_solution,
#             "rating": user_rating,  # 1-5 scale
#             "comments": user_comments,
#             "timestamp": pd.Timestamp.now()
#         }
#         self.feedback_data.append(feedback_entry)
#         print(f"Collected feedback: Rating {user_rating}/5")
    
#     def create_optimization_dataset(self):
#         """Create dataset for DSPy optimization from feedback"""
#         if not self.feedback_data:
#             return []
        
#         # Convert feedback to DSPy examples
#         examples = []
#         for feedback in self.feedback_data:
#             if feedback["rating"] >= 4:  # Good solutions
#                 example = dspy.Example(
#                     question=feedback["question"],
#                     solution=feedback["generated_solution"]
#                 ).with_inputs("question")
#                 examples.append(example)
        
#         return examples
    
#     def optimize_with_feedback(self):
#         """Optimize the RAG module using collected feedback"""
#         examples = self.create_optimization_dataset()
        
#         if len(examples) < 3:
#             print("Need at least 3 good examples for optimization")
#             return
        
#         # Define metric for optimization
#         def solution_quality_metric(example, prediction, **kwargs):
#             """Custom metric to evaluate solution quality"""
#             # Simple keyword-based evaluation
#             solution = prediction.solution.lower()
#             quality_score = 0
            
#             # Check for step-by-step structure
#             if "step" in solution:
#                 quality_score += 0.3
            
#             # Check for mathematical reasoning indicators
#             math_indicators = ["therefore", "solution", "answer", "calculate", "solve"]
#             for indicator in math_indicators:
#                 if indicator in solution:
#                     quality_score += 0.1
            
#             # Length-based scoring
#             if 100 <= len(solution) <= 1000:
#                 quality_score += 0.3
            
#             return min(quality_score, 1.0)
        
#         # Optimize using DSPy BootstrapFewShot
#         try:
#             # Split examples for training and validation
#             split_point = len(examples) // 2
#             trainset = examples[:split_point] if split_point > 0 else examples
#             valset = examples[split_point:] if split_point > 0 and len(examples) > 1 else examples[:1]
            
#             optimizer = dspy.BootstrapFewShot(metric=solution_quality_metric)
#             optimized_module = optimizer.compile(
#                 self.rag_module, 
#                 trainset=trainset, 
#                 valset=valset
#             )
#             self.rag_module = optimized_module
#             print("Successfully optimized RAG module with feedback")
#         except Exception as e:
#             print(f"Optimization failed: {e}")
    
#     def display_feedback_summary(self):
#         """Display summary of collected feedback"""
#         if not self.feedback_data:
#             print("No feedback collected yet")
#             return
        
#         df = pd.DataFrame(self.feedback_data)
#         print(f"\nFeedback Summary:")
#         print(f"Total feedback entries: {len(self.feedback_data)}")
#         print(f"Average rating: {df['rating'].mean():.2f}")
#         print(f"Rating distribution:")
#         print(df['rating'].value_counts().sort_index())

# # Initialize DSPy optimizer
# dspy_optimizer = DSPyMathOptimizer()



In [ ]:
# # Add more test problems and feedback
# problems = [
#     "What is the integral of 2x dx?",
#     "Solve for x: 2x + 5 = 15", 
#     "What is the area of a circle with radius 3?"
# ]

# for problem in problems:
#     solution = dspy_optimizer.solve_problem(problem)
#     print(f"Problem: {problem}")
#     print(f"Solution: {solution}")
#     # Simulate positive feedback
#     dspy_optimizer.collect_feedback(problem, solution, 4, "Clear solution")
#     print("Collected feedback: Rating 4/5")
#     print()

# # Try optimization
# print("Attempting optimization...")
# dspy_optimizer.optimize_with_feedback()



Problem: What is the integral of 2x dx?
Solution: x² + C
Collected feedback: Rating 4/5
Collected feedback: Rating 4/5

Problem: Solve for x: 2x + 5 = 15
Solution: x = 5
Collected feedback: Rating 4/5
Collected feedback: Rating 4/5



2025/09/23 13:00:46 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Problem: What is the area of a circle with radius 3?
Solution: 9π
Collected feedback: Rating 4/5
Collected feedback: Rating 4/5

Attempting optimization...
Optimization failed: BootstrapFewShot.compile() got an unexpected keyword argument 'valset'


In [ ]:
# # Check total feedback collected
# dspy_optimizer.display_feedback_summary()

# # Test if optimization improved performance
# test_question = "What is the derivative of 3x² + 2x?"
# optimized_solution = dspy_optimizer.solve_problem(test_question)
# print(f"Optimized solution: {optimized_solution}")



Feedback Summary:
Total feedback entries: 6
Average rating: 4.00
Rating distribution:
rating
4    6
Name: count, dtype: int64
Optimized solution: 6x + 2


In [ ]:
import os
import dspy
import litellm
import pandas as pd

class DSPyMathOptimizer:
    """DSPy-based optimizer for math education with enhanced human feedback"""
    
    def __init__(self):
        # Configure DSPy with Groq using your working setup
        self.lm = dspy.LM(
            model="groq/moonshotai/kimi-k2-instruct",
            api_key=os.environ.get("GROQ_API_KEY")
        )
        dspy.configure(lm=self.lm)
        
        # Initialize components
        self.rag_module = self._create_rag_module()
        self.feedback_data = []
        
    def _create_rag_module(self):
        """Create DSPy RAG module for math problems"""
        
        class MathRAG(dspy.Module):
            def __init__(self):
                super().__init__()
                self.generate_answer = dspy.ChainOfThought("question, context -> solution")
            
            def forward(self, question, context=""):
                prediction = self.generate_answer(question=question, context=context)
                return dspy.Prediction(solution=prediction.solution)
        
        return MathRAG()
    
    def solve_problem(self, question: str, context: str = ""):
        """Solve a math problem using the RAG module"""
        try:
            result = self.rag_module(question=question, context=context)
            return result.solution
        except Exception as e:
            print(f"Error solving problem: {e}")
            return "Unable to solve problem"
    
    def collect_detailed_feedback(self, question: str, generated_solution: str, 
                                user_rating: int, detailed_comments: dict):
        """Collect comprehensive human feedback with detailed descriptions"""
        feedback_entry = {
            "question": question,
            "generated_solution": generated_solution,
            "rating": user_rating,  # 1-5 scale
            "detailed_comments": detailed_comments,
            "timestamp": pd.Timestamp.now()
        }
        self.feedback_data.append(feedback_entry)
        print(f" Collected detailed feedback: Rating {user_rating}/5")
        
        # Display the detailed feedback
        print(" Detailed Feedback:")
        for aspect, comment in detailed_comments.items():
            print(f"   • {aspect}: {comment}")
    
    def create_optimization_dataset(self):
        """Create dataset for DSPy optimization from feedback"""
        if not self.feedback_data:
            return []
        
        # Convert feedback to DSPy examples
        examples = []
        for feedback in self.feedback_data:
            if feedback["rating"] >= 4:  # Good solutions
                example = dspy.Example(
                    question=feedback["question"],
                    solution=feedback["generated_solution"]
                ).with_inputs("question")
                examples.append(example)
        
        return examples
    
    def optimize_with_feedback(self):
        """Optimize the RAG module using collected feedback"""
        examples = self.create_optimization_dataset()
        
        if len(examples) < 2:
            print(f"Need at least 2 good examples for optimization. Currently have {len(examples)}")
            return
        
        print(f" Starting optimization with {len(examples)} examples...")
        
        # Define enhanced metric using detailed feedback
        def solution_quality_metric(example, prediction, **kwargs):
            """Enhanced metric incorporating detailed feedback patterns"""
            try:
                solution = str(prediction.solution).lower()
                quality_score = 0.3  # Base score
                
                # Check for mathematical reasoning indicators from feedback
                quality_indicators = [
                    "step", "therefore", "solution", "answer", "calculate", 
                    "derivative", "integral", "equation", "formula", "method"
                ]
                for indicator in quality_indicators:
                    if indicator in solution:
                        quality_score += 0.1
                
                # Length-based scoring (from detailed feedback patterns)
                if 50 <= len(solution) <= 800:
                    quality_score += 0.2
                
                return min(quality_score, 1.0)
            except:
                return 0.0
        
        # Optimize using DSPy BootstrapFewShot
        try:
            optimizer = dspy.BootstrapFewShot(
                metric=solution_quality_metric,
                max_bootstrapped_demos=3,
                max_labeled_demos=2,
                max_rounds=1,
                max_errors=5
            )
            
            optimized_module = optimizer.compile(
                self.rag_module, 
                trainset=examples
            )
            self.rag_module = optimized_module
            print(" Successfully optimized RAG module with detailed feedback")
        except Exception as e:
            print(f" Optimization failed: {e}")
    
    def display_detailed_feedback_summary(self):
        """Display comprehensive summary of collected feedback"""
        if not self.feedback_data:
            print("No feedback collected yet")
            return
        
        df = pd.DataFrame(self.feedback_data)
        print(f"\n Detailed Feedback Summary:")
        print(f"Total feedback entries: {len(self.feedback_data)}")
        print(f"Average rating: {df['rating'].mean():.2f}")
        print(f"Rating distribution:")
        print(df['rating'].value_counts().sort_index())
        
        # Analyze detailed comments
        print(f"\n Detailed Comment Analysis:")
        all_aspects = set()
        for feedback in self.feedback_data:
            if 'detailed_comments' in feedback:
                all_aspects.update(feedback['detailed_comments'].keys())
        
        for aspect in all_aspects:
            print(f"\n    {aspect.upper()}:")
            for i, feedback in enumerate(self.feedback_data[-3:], 1):  # Show last 3
                if 'detailed_comments' in feedback and aspect in feedback['detailed_comments']:
                    print(f"      {i}. {feedback['detailed_comments'][aspect]}")

# Initialize enhanced DSPy optimizer
dspy_optimizer = DSPyMathOptimizer()

# Test with detailed feedback collection
print(" Testing Enhanced Feedback Collection System")

# Test problems with detailed feedback
test_cases = [
    {
        "question": "What is the derivative of x^2 + 3x + 2?",
        "feedback": {
            "rating": 5,
            "comments": {
                "clarity": "Excellent step-by-step breakdown with clear explanations",
                "accuracy": "Mathematically correct using power rule properly",
                "completeness": "Includes all necessary steps and final answer",
                "pedagogy": "Great teaching approach, easy to follow for students"
            }
        }
    },
    {
        "question": "What is the integral of 2x dx?",
        "feedback": {
            "rating": 4,
            "comments": {
                "clarity": "Good explanation but could use more detail on constant of integration",
                "accuracy": "Correct application of integration rules",
                "completeness": "Missing brief explanation of why +C is needed",
                "pedagogy": "Solid approach, would benefit from mentioning antiderivative concept"
            }
        }
    },
    {
        "question": "Solve for x: 2x + 5 = 15",
        "feedback": {
            "rating": 5,
            "comments": {
                "clarity": "Perfect step-by-step algebraic manipulation",
                "accuracy": "Correct isolation of variable and arithmetic",
                "completeness": "Shows all work including verification step",
                "pedagogy": "Excellent for teaching basic equation solving techniques"
            }
        }
    }
]

# Process test cases with detailed feedback
for case in test_cases:
    question = case["question"]
    solution = dspy_optimizer.solve_problem(question)
    
    print(f"\n Problem: {question}")
    print(f" Solution: {solution}")
    
    # Collect detailed feedback
    dspy_optimizer.collect_detailed_feedback(
        question=question,
        generated_solution=solution,
        user_rating=case["feedback"]["rating"],
        detailed_comments=case["feedback"]["comments"]
    )
    print("-" * 80)

# Display comprehensive feedback analysis
dspy_optimizer.display_detailed_feedback_summary()

# Try optimization with enhanced feedback
print("\n🔧 Attempting optimization with detailed feedback...")
dspy_optimizer.optimize_with_feedback()


🎯 Testing Enhanced Feedback Collection System


2025/09/23 13:09:50 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.



📚 Problem: What is the derivative of x^2 + 3x + 2?
💡 Solution: 2x + 3
✅ Collected detailed feedback: Rating 5/5
📝 Detailed Feedback:
   • clarity: Excellent step-by-step breakdown with clear explanations
   • accuracy: Mathematically correct using power rule properly
   • completeness: Includes all necessary steps and final answer
   • pedagogy: Great teaching approach, easy to follow for students
--------------------------------------------------------------------------------

📚 Problem: What is the integral of 2x dx?
💡 Solution: x² + C
✅ Collected detailed feedback: Rating 4/5
📝 Detailed Feedback:
   • clarity: Good explanation but could use more detail on constant of integration
   • accuracy: Correct application of integration rules
   • completeness: Missing brief explanation of why +C is needed
   • pedagogy: Solid approach, would benefit from mentioning antiderivative concept
--------------------------------------------------------------------------------

📚 Problem: Solve for 

100%|██████████| 3/3 [00:10<00:00,  3.64s/it]

Bootstrapped 0 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.
✅ Successfully optimized RAG module with detailed feedback
